## **Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

     |████████████████████████████████| 11.0MB 6.2MB/s 
     |████████████████████████████████| 747kB 35.2MB/s 
     |████████████████████████████████| 1.7MB 2.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 275kB/s 
     |████████████████████████████████| 9.9MB 38.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=5d5880426fe2d020240e4f73e90efef88676ed96b886e0aa6ac49bec4422bda0
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0

In [9]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
df = pd.read_csv('Wongnai Reviews - Small.csv')

In [3]:
df.tail()

,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


## **Tokenize Words**

In [4]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [5]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [6]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


## **Create Dictionary**

In [10]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [11]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [12]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

## **Topic Modeling**

In [13]:
num_topics = 30
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 14.2 s, sys: 11.3 s, total: 25.5 s
Wall time: 13.1 s


In [15]:
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.070657  0.090787       1        1  13.270156
4      0.042413 -0.053405       2        1   8.134347
17    -0.006040 -0.045293       3        1   7.222248
3     -0.007819 -0.017569       4        1   7.200658
10     0.126469  0.019611       5        1   7.110414
13    -0.019605  0.026586       6        1   5.705314
0      0.060227 -0.035891       7        1   5.557571
12    -0.022420 -0.052412       8        1   4.779858
28    -0.048431  0.111464       9        1   4.297902
23     0.010062 -0.018790      10        1   3.671253
22     0.018882 -0.002568      11        1   3.551854
1     -0.051306  0.000442      12        1   3.452616
2      0.034088 -0.008616      13        1   2.816076
15     0.021826 -0.002080      14        1   2.581846
14    -0.047590 -0.005033      15        1   2.466728
11    -0.048092 -0.001399      16        1   2.399721
7     -0.008359  0.005254      17        1   2.263133
18     0.000435  0.008175      18        1   2.192948
21    -0.040233  0.005779      19        1   1.899401
26    -0.012839 -0.013331      20        1   1.718451
9     -0.024805 -0.016809      21        1   1.541596
20    -0.009329 -0.011408      22        1   1.472790
8      0.009576 -0.002345      23        1   1.236032
16    -0.012315 -0.001879      24        1   0.955326
27    -0.017599  0.010280      25        1   0.802044
25    -0.001892 -0.003295      26        1   0.744944
19    -0.008425  0.008773      27        1   0.499993
29    -0.007413  0.001811      28        1   0.272093
5      0.000849  0.000286      29        1   0.170431
24    -0.000972  0.002876      30        1   0.012255, topic_info=       Term        Freq       Total Category  logprob  loglift
157   อร่อย  472.000000  472.000000  Default  30.0000  30.0000
2      กาแฟ  291.000000  291.000000  Default  29.0000  29.0000
46   รสชาติ  384.000000  384.000000  Default  28.0000  28.0000
11      ทาน  382.000000  382.000000  Default  27.0000  27.0000
39       ดี  343.000000  343.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
40     นั่ง    0.017140  190.274495  Topic30  -5.5286  -0.3078
36       ชา    0.019027  224.419258  Topic30  -5.4241  -0.3684
438      นม    0.012280  130.107222  Topic30  -5.8620  -0.2612
88      จาน    0.013106  179.648721  Topic30  -5.7969  -0.5187
171    เค้า    0.011388  122.948069  Topic30  -5.9374  -0.2799

[2188 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
1678     17  0.420524                                                   
1678     19  0.420524                                                   
1475      2  0.233743                                                   
1475      6  0.233743                                                   
1475     19  0.233743                                                   
...     ...       ...                                                ...
4693      3  0.900675                                                 ••
3148     24  0.743569  …................................................
2129     16  0.568542                                                  ′
2130     16  0.568899                                                  ″
4010     13  0.862433                                               ⭐⭐⭐⭐

[5071 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 18, 4, 11, 14, 1, 13, 29, 24, 23, 2, 3, 16, 15, 12, 8, 19, 22, 27, 10, 21, 9, 17, 28, 26, 20, 30, 6, 25])

In [16]:
model.show_topic(1)

[('ทาน', 0.016074752),
 ('ชา', 0.015533653),
 ('กาแฟ', 0.012792304),
 ('แตงโม', 0.012031745),
 ('ดี', 0.0101871),
 ('บาท', 0.009787048),
 ('ลอง', 0.00970984),
 ('อร่อย', 0.009707504),
 ('รสชาติ', 0.008986267),
 ('ไข่มุก', 0.0084979795)]

In [17]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [18]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",6,0.785833
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",3,0.486436
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",28,0.989259
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",4,0.069041
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...",4,0.934360
